Source: https://towardsdatascience.com/scikit-learn-pipeline-tutorial-with-parameter-tuning-and-cross-validation-e5b8280c01fb

### Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import RandomizedSearchCV

### Load and Preprocess Data

In [ ]:
data = pd.read_csv('../Data/bank-full.csv', sep=';')
target = data.pop('y')
target = target.map({'yes': 1, 'no':0})
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=1234)

In [ ]:
# mask for categorical columns
categorical_mask = (data.dtypes=='object')
# get a list of categorical columns
categorical_columns = data.columns[categorical_mask].tolist()
# get a list of numeric columns
num_cols = data.select_dtypes(include=['int64','float64']).columns.tolist()
# columns to apply OrdinalEncoder on.
oe_cols = [c for c in categorical_columns if data[c].nunique()>5]
# columns to apply OneHotEncoder on.
ohe_cols = [c for c in categorical_columns if data[c].nunique()<=5]
len(oe_cols), len(ohe_cols), len(num_cols)

In [ ]:
## Prepare encoders and Imputer
ohe_unique_list = [data[c].unique().tolist() for c in ohe_cols]
oe_unique_list = [data[c].unique().tolist() for c in oe_cols]
ohe = OneHotEncoder(categories=ohe_unique_list)
oe = OrdinalEncoder(categories=oe_unique_list)
imp = SimpleImputer(strategy='constant', fill_value=0)

In [ ]:
## put them all in a make_column_transformer
# https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_transformer.html
preprocess = make_column_transformer(
    (oe, oe_cols),
    (ohe, ohe_cols),
    (imp, num_cols),
    remainder='passthrough'
)

### Prepare Pipeline with Preprocessing, Learner and Feature Selection

In [ ]:
estimator = RandomForestClassifier()
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html
# Compute the ANOVA F-value for the provided sample.
fs = SelectKBest(score_func=f_classif, k=5)
#selector = RFE(estimator, n_features_to_select=5, step=1)
steps = [
    ('preprocess', preprocess),
    ('select', fs),
    ('clf', estimator)
]
pipeline = Pipeline(steps)

### Fit the Pipeline
Now, It’s as simple as any other machine learning algorithm, we first fit and then use predict. Predict function does all other preprocessing and then applies the trained model.

In [ ]:
pipeline.fit(X_train, y_train)
#y_pred = pipeline.predict(X_test)
y_pred = pipeline.predict_proba(X_test)[:,1]

pred_df = pd.DataFrame({'y': y_test,'y_pred': y_pred})
print(roc_auc_score(y_test, y_pred))

## Randomized Search with CV

In [ ]:
param_grid = {
    'clf__max_depth': np.arange(3,10,1),
    'clf__n_estimators': np.arange(50,250,50)
}

In [ ]:
rand_auc = RandomizedSearchCV(estimator=pipeline, param_distributions=param_grid, n_iter=5, scoring='roc_auc', cv=5, verbose=False)
rand_auc.fit(X_train, y_train)
print(rand_auc.best_score_)

In [ ]:
y_pred = rand_auc.predict_proba(X_test)[:,1]
pred_df = pd.DataFrame({'y': y_test,'y_pred': y_pred})
print(roc_auc_score(y_test, y_pred))

## Evaluate Multiple Classifiers (See How Easy it is!)
Takes a couple of minutes!

In [ ]:
%%time
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),    
    RandomForestClassifier(),
    GradientBoostingClassifier()
    ]

for classifier in classifiers:
    steps = [
        ('preprocess', preprocess),
        ('select', fs),
        ('clf', classifier)
    ]
    pipeline = Pipeline(steps)
    pipeline.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipeline.score(X_test, y_test))

As you can see, using scikit-learn’s Pipeline feature helps a lot in streamlining machine learning workflow and makes a data scientist's job easier and can focus their time on fine-tuning models, rather than doing data pre-processing steps repetitively.